In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [11]:
# Load the data
df = pd.read_csv(r'C:\Users\honey\Documents\placment work\Electronical-AI-Device-Recogniser\khanya\data managment\acs-f2-dataset.csv')


In [12]:

# Data Preprocessing
# Drop the 'time' column as it's not relevant for prediction
df = df.drop('time', axis=1)

In [13]:
# Convert categorical labels to numeric
le = LabelEncoder()
df['equipment'] = le.fit_transform(df['equipment'])


In [14]:
# Split features and labels
X = df.drop('equipment', axis=1)
y = df['equipment']


In [15]:
# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [16]:
# Handle class imbalance using SMOTE
from imblearn.over_sampling import SMOTE
from tensorflow.keras.utils import to_categorical

# Apply SMOTE to create a balanced dataset
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X, y)

# One-hot encode the target labels
num_classes = len(np.unique(y))  # Number of output classes
y_resampled_onehot = to_categorical(y_resampled, num_classes=num_classes)


In [17]:

# Build an improved neural network model with more layers and dropout for regularization
model = Sequential()

# Input layer
model.add(Dense(128, input_dim=X.shape[1], activation='relu'))

# Hidden layers with dropout
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))  # Dropout for regularization

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))  # Dropout for regularization

# Output layer
model.add(Dense(len(np.unique(y)), activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with validation
history = model.fit(X_resampled, y_resampled_onehot, epochs=50, validation_split=0.2)


Epoch 1/50


c:\Users\honey\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3631/3631 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.1735 - loss: 3.3769 - val_accuracy: 0.1160 - val_loss: 6.7266
Epoch 2/50
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 4s 982us/step - accuracy: 0.2636 - loss: 2.1042 - val_accuracy: 0.1067 - val_loss: 9.0241
Epoch 3/50
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 4s 996us/step - accuracy: 0.2981 - loss: 2.0210 - val_accuracy: 0.1133 - val_loss: 10.2730
Epoch 4/50
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 4s 992us/step - accuracy: 0.3551 - loss: 1.8897 - val_accuracy: 0.1289 - val_loss: 12.7819
Epoch 5/50
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 4s 978us/step - accuracy: 0.3809 - loss: 1.8164 - val_accuracy: 0.1270 - val_loss: 13.8524
Epoch 6/50
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.3908 - loss: 1.7935 - val_accuracy: 0.1210 - val_loss: 16.6151
Epoch 7/50
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 4s 990us/step - accuracy: 0.3982 - loss: 1.7709 - val_accuracy: 0.1304 - val_loss: 22.3982
Epoch 8/50
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 4s 975us/step - accuracy: 0.4042 - loss: 1.7

In [9]:

# Evaluate model performance
from sklearn.metrics import classification_report, confusion_matrix

# Predict on the validation set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print(conf_matrix)

# Classification report
print(classification_report(y_test, y_pred_classes, target_names=le.classes_))


NameError: name 'X_test' is not defined

In [35]:
# Convert labels to one-hot encoding
y_encoded = to_categorical(y)


In [36]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [ ]:
# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(y_encoded.shape[1], activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, verbose=1)

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

In [ ]:
# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Plot confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.show()

In [15]:
# Function to predict appliance and probabilities
def predict_appliance(electrical_readings):
    # Ensure the input is a 2D array
    if electrical_readings.ndim == 1:
        electrical_readings = electrical_readings.reshape(1, -1)
    
    # Scale the input
    scaled_input = scaler.transform(electrical_readings)
    
    # Make prediction
    probabilities = model.predict(scaled_input)[0]
    predicted_class = le.inverse_transform([np.argmax(probabilities)])[0]
    
    return predicted_class, probabilities

In [ ]:
# Example usage
example_reading = np.array([50.0, 340, 111.284, -40.89, 0.533, 224.038])
predicted_appliance, probabilities = predict_appliance(example_reading)

print(f"Predicted appliance: {predicted_appliance}")
print("Probabilities for each appliance:")
for appliance, prob in zip(le.classes_, probabilities):
    print(f"{appliance}: {prob:.4f}")


In [ ]:

# Plot probabilities
plt.figure(figsize=(10, 6))
sns.barplot(x=le.classes_, y=probabilities)
plt.title('Appliance Prediction Probabilities')
plt.xlabel('Appliance')
plt.ylabel('Probability')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [18]:
import re
import joblib
import os

def get_next_version(file_name):
    # Find version pattern like 1.1.2.3
    match = re.search(r'(\d+\.\d+\.\d+\.\d+)', file_name)
    if match:
        version_str = match.group(1)
        version_parts = list(map(int, version_str.split('.')))
        version_parts[-1] += 1  # Increment the last number
        next_version = '.'.join(map(str, version_parts))
        return file_name.replace(version_str, next_version)
    else:
        raise ValueError("No version found in file name")

# Example for scaler
scaler_file = 'scaler1.1.2.3.pkl'
next_scaler_file = get_next_version(scaler_file)
joblib.dump(scaler, next_scaler_file)
print(f"Saved scaler as: {next_scaler_file}")

# Example for model
model_file = 'appliance_recogniser#1.1.3.keras'
next_model_file = get_next_version(model_file)
model.save(next_model_file)
print(f"Saved model as: {next_model_file}")
